# The Community Coronavirus

Data Sources:

1. [Our World In Data (Global)](https://ourworldindata.org/coronavirus)
1. [Minister of Health Malaysia (Local)](https://github.com/MoH-Malaysia/covid19-public)

The notebook cover two different perspective, the local view (Malaysia) and global view (World).<br>
`Just COVID-19`<br>
For local -> [Local view](#local_view)<br>
& global -> [Global](#global_view)<br>

Implemented:

1. Simulating pandemic with SEIR compartment model. [Navigate](#seir_model)
1. Using logistic function to identify inflection point and potential growth. [Navigate](#logistic_function)
1. Exploratory data analysis.
1. Forward Forecasting with ensemble machine learning model, stacked model and LSTM model. [Navigate](#predicting_section)

[Dependencies](#dependencies)

![title](https://upload.wikimedia.org/wikipedia/commons/9/94/Coronavirus._SARS-CoV-2.png)

## Local<a id='local_view'></a>

### Import

In [ ]:
import warnings

import lightgbm as lgb
import matplotlib as mpl
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import xgboost as xgb
from cycler import cycler
from mlxtend.regressor import StackingCVRegressor
from numba import jit
from scipy.integrate import solve_ivp
from scipy.optimize import curve_fit, minimize
from sklearn import *
from sklearn import base

warnings.filterwarnings('ignore')

custom_style = {
    'figure.autolayout': True,
    'figure.titlesize': 20,
    'figure.figsize': (10, 5),
    'figure.dpi': 100,
    'axes.spines.top': False,
    'axes.spines.left': False,
    'axes.titlesize': 10,
    'axes.titlelocation': 'left',
    'axes.labelsize': 14,
    'axes.grid': True,
    'axes.prop_cycle': cycler(
        color=['#d73027', '#00518b', '#b1ef89', '#ffd500', '#000000']
    ),
    'grid.color': '#969696',
    'xtick.direction': 'inout',
    'ytick.direction': 'inout',
    'xtick.minor.visible': True,
    'ytick.minor.visible': True,
    'ytick.right': True,
    'ytick.left': False,
    'ytick.labelright': True,
    'ytick.labelleft': False,
    'xaxis.labellocation': 'right',
    'yaxis.labellocation': 'top',
    'font.family': 'monospace',
    'legend.fontsize': 10,
    'legend.loc': 'best',
}

custom_style2 = {
    'xtick.major.size': 7,
    'xtick.minor.size': 3.5,
    'xtick.major.width': 1.1,
    'xtick.minor.width': 1.1,
    'xtick.major.pad': 5,
    'xtick.minor.visible': True,
    'xtick.top': False,
    'xtick.labelsize': 12,
    'ytick.major.size': 7,
    'ytick.minor.size': 3.5,
    'ytick.major.width': 1.1,
    'ytick.minor.width': 1.1,
    'ytick.major.pad': 5,
    'ytick.minor.visible': True,
    'ytick.right': True,
    'ytick.labelsize': 12,
    'font.family': 'Palatino Linotype',
    'font.size': 16,
    'ytick.right': True,
    'ytick.left': False,
    'ytick.labelright': True,
    'ytick.labelleft': False,
    'xaxis.labellocation': 'right',
    'yaxis.labellocation': 'top',
    'axes.spines.top': False,
    'axes.spines.left': False,
}

# %config InlineBackend.figure_format='svg'
plt.style.use(custom_style)

### Preprocessing

In [ ]:
%%time
owid = pd.read_csv("https://covid.ourworldindata.org/data/owid-covid-data.csv")
owid = owid[owid['location'] == 'Malaysia'][
    [
        'date',
        'total_cases',
        'new_cases',
        'total_deaths',
        'new_deaths',
        'total_tests',
        'new_tests',
        'total_vaccinations',
        'new_vaccinations',
    ]
]
owid.columns = [
    'date',
    'total_cases',
    'new_cases',
    'total_deceased',
    'new_deceased',
    'total_tests',
    'new_tests',
    'total_vaccinations',
    'new_vaccinations',
]
# owid = owid.iloc[70:-14]
owid['date'] = pd.to_datetime(owid['date'])
owid.set_index('date', inplace=True)
assert type(owid.index) == pd.core.indexes.datetimes.DatetimeIndex

moh = pd.read_csv(
    "https://raw.githubusercontent.com/MoH-Malaysia/covid19-public/main/epidemic/cases_malaysia.csv"
)
moh = moh[['date', 'cases_recovered', 'cases_active']]
moh.rename(
    columns={'cases_recovered': 'new_recovered', 'cases_active': 'new_active'},
    inplace=True,
)
moh['total_recovered'] = moh['new_recovered'].cumsum()
moh['total_active'] = moh['new_active'].cumsum()
moh['date'] = pd.to_datetime(moh['date'])
moh.set_index('date', inplace=True)
assert type(moh.index) == pd.core.indexes.datetimes.DatetimeIndex

data = pd.merge(owid, moh, how='left', left_index=True, right_index=True)
data.columns = [
    'total_cases',
    'cases',
    'total_deceased',
    'deceased',
    'total_tests',
    'tests',
    'total_vaccinations',
    'vaccinations',
    'recovered',
    'active',
    'total_recovered',
    'total_active',
]
data = data.loc['2020-02-02':'2022-02-02']
data.fillna(0, inplace=True)
fdata = data.reset_index(drop=True)
fdata

### SEIR Model<a id='seir_model'></a>

SEIR model, a compartment model for modelling infectious diseases spread where the total population is assigned to either Susceptible, Exposed, Infectious and Recovered.

In [ ]:
skipped_window = 21
total_infected = data['total_cases'][skipped_window:]
total_deceased = data['total_deceased'][skipped_window:]
total_recovered = data['total_recovered'][skipped_window:]


@jit(nopython=True)
def SEIR(t, y, beta, gamma, sigma, alpha, t_quarantine):

    N = np.int(33e6 / (10 / 0.55))

    S = y[0]  # Susceptible
    E = y[1]  # Exposed
    I = y[2]  # Infected
    R = y[3]  # Recovered

    if t > t_quarantine:
        beta_t = beta * np.exp(-alpha * (t - t_quarantine))
    else:
        beta_t = beta

    dS = -beta_t * S * I / N
    dE = beta_t * S * I / N - sigma * E
    dI = sigma * E - gamma * I
    dR = gamma * I

    return [dS, dE, dI, dR]


def fitting_SEIR(vec, t_q, N, test_size):
    beta, gamma, sigma, alpha = vec

    t_f = total_infected.shape[0]
    y0 = [N - total_infected[0], 0, total_infected[0], 0]
    t_eval = np.arange(0, t_f, 1)
    N = np.int(33e6 / (10 / 0.55))

    sol = solve_ivp(
        SEIR, [0, t_f], y0, args=(beta, gamma, sigma, alpha, t_q), t_eval=t_eval
    )

    split = np.int((1 - test_size) * total_infected.shape[0])

    error = (
        np.sum(
            5
            * (total_deceased[:split] + total_recovered[:split] - sol.y[3][:split]) ** 2
        )
        + np.sum(
            (total_infected[:split] - np.cumsum(sol.y[1][:split] + sol.y[2][:split]))
            ** 2
        )
    ) / split

    return error


def interpret_SEIR(plot=True):
    N = np.int(33e6 / (10 / 0.55))
    t_q = 21
    t_f = total_infected.shape[0]
    y0 = [N - total_infected[0], 0, total_infected[0], 0]
    t_eval = np.arange(0, t_f, 1)
    test_size = 0.1
    opts = minimize(
        fitting_SEIR, [2, 1, 0.8, 0.3], method='Nelder-Mead', args=(t_q, N, test_size)
    )
    beta, gamma, sigma, alpha = opts.x
    sol = solve_ivp(
        SEIR, [0, t_f], y0, args=(beta, gamma, sigma, alpha, t_q), t_eval=t_eval
    )

    if plot:
        fig, ax = plt.subplots(figsize=(10, 5), dpi=200)
        ax.plot(
            data.index[skipped_window:],
            np.cumsum(sol.y[1] + sol.y[2]),
            label='Exposed and Infected',
        )
        ax.plot(data.index[skipped_window:], total_infected, label='Infected')
        ax.plot(data.index[skipped_window:], sol.y[3], label='Recovered')
        plt.suptitle(
            'SEIR Model: An Assumption', ha='left', x=0.015, y=0.95, fontsize=20
        )
        plt.legend()
        plt.xlabel('Date')
        plt.ylabel('Population')
        ax.yaxis.set_label_position('right')
        plt.title(
            'Skipped Window: {} | Quarantine Take Place {} days after outbreak'.format(
                skipped_window, t_q
            )
        )
        plt.savefig("fig/seir_model.png")
        plt.show()


interpret_SEIR(plot=True)

### Exploratory

In [ ]:
# Helper function
def percent_change(df, target_columns):
    assert type(df) == pd.core.frame.DataFrame
    assert type(target_columns) == str
    percent_change = df[target_columns].pct_change() * 100
    return percent_change


def absolute_change(df, target_columns):
    assert type(df) == pd.core.frame.DataFrame
    assert type(target_columns) == str
    absolute_change = df[target_columns] - df[target_columns].shift(1)
    return absolute_change


def style_negative(v, props=""):
    return props if v < 0 else None


def style_positive(v, props=""):
    return props if v > 0 else None

In [ ]:
def changes_in_series(df):
    data = df.copy()
    print("Stats In a Glance")
    needed = ['tests', 'cases', 'deceased', 'vaccinations', 'recovered']
    for c in needed:
        data[c + ' Percent Change'] = percent_change(data, c)

    percent_change_cols = [col for col in data.columns if 'Percent Change' in col]
    display(
        data[percent_change_cols]
        .tail(20)
        .style.applymap(style_negative, props='color:red;')
        .applymap(style_positive, props='color:green;')
        .format(na_rep='Missing')
    )


changes_in_series(df=data)

### Logistic Function<a id='logistic_function'></a>

Additional to SEIR general mathematic model, Logistic Function can be also used to estimate the inflection point and the overall trend of the pandemic.

In [ ]:
def logistic_function(x, k, x_0, maxy):
    return maxy / (
        1 + np.exp(-k * (x - x_0))
    )  # k = growth rate | x_0 = inflection point | maxy = the curve maximum value


def logistic_pred(df, cn, initial_start, plot=True):
    assert type(cn) == str

    truey = df[cn][initial_start:]
    data_length = range(truey.shape[0])
    popt, pcov = curve_fit(
        logistic_function, data_length, truey, bounds=([0, 0, 0], np.inf), maxfev=2000
    )
    estimated_k, estimated_x_0, maxy = popt
    predy = logistic_function(data_length, estimated_k, estimated_x_0, maxy)
    mse = np.square(np.subtract(truey, predy)).mean()

    if plot:
        fig, ax = plt.subplots(figsize=(10, 5), dpi=200)
        ax.plot(data_length, predy, 'r--', label='Predict', linewidth=2)
        ax.plot(data_length, truey, color='black', label='Actual', linewidth=2)
        ax.legend(fontsize='large')
        ax.set_xlabel(
            'Logistic Function Error vs Actual | MSE: {}'.format(round(mse, 4)),
            fontsize=10,
        )
        ax.set_ylabel('Values')
        ax.yaxis.set_label_position('right')
        plt.suptitle(
            'Logistic Function on Inflection Point & Growth Rate',
            ha='left',
            x=0.015,
            y=0.95,
            fontsize=20,
        )
        ax.set_title(
            'Estimated Growth Rate: {} | Estimated Inflection Point: {} | Estimated Maximum Cases: {}'.format(
                round(estimated_k, 4), round(estimated_x_0, 4), round(maxy, 4)
            )
        )
        plt.savefig("fig/logistic_growth.png")
        plt.show()


logistic_pred(df=fdata, cn='cases', initial_start=0, plot=True)

In [ ]:
def condition_color_list(columns):
    color_list = []
    for v in columns:
        if v > 0:
            color_list.append('#15607a')
        else:
            color_list.append('#ff483a')
    return color_list


def plot_pc(df):
    mpl.rcParams.update(mpl.rcParamsDefault)
    with plt.style.context(custom_style2):
        plt.style.use(['dark_background'])
        temp = df.copy()
        temp['Total Confirm Percent Change'] = percent_change(temp, 'cases')

        color_list = condition_color_list(temp['Total Confirm Percent Change'])
        fig, ax = plt.subplots(figsize=(10, 5), dpi=200)
        plt.tight_layout()
        ax.bar(
            x=temp.index, height=temp['Total Confirm Percent Change'], color=color_list
        )
        ax.plot(
            temp.index,
            [np.mean(temp['Total Confirm Percent Change'])] * len(temp.index),
            label='Mean',
            linestyle='--',
            color='#ffb55f',
        )
        ax.yaxis.set_label_position('right')
        ax.grid(axis='y')
        ax.legend(fontsize=10)
        ax.set_xlabel('Year')
        ax.set_ylabel('Percent Change')
        ax.set_title('Covid-19 Confirmed Percent Change', fontweight='bold', loc='left')
        ax.set_ylim([-100, 500])
        print(
            'Max: {}, Min: {}'.format(
                temp['Total Confirm Percent Change'].max(),
                temp['Total Confirm Percent Change'].min(),
            )
        )
        plt.savefig("fig/percent_change.png")
        plt.show()


plot_pc(df=data)

In [ ]:
mpl.rcParams.update(mpl.rcParamsDefault)
plt.style.use(custom_style)

In [ ]:
def plot_decomposition(df, cn, window, plot=True):
    assert type(cn) == str

    values = df[cn].values.flatten()
    rm = np.convolve(values, np.ones(window) / window, mode='valid')
    detrended = values[window - 1 :] / rm
    diff = df[cn] - df[cn].shift(1)

    new = df.reset_index().copy()
    new['day'] = new['date'].dt.day_name()
    countday = new.groupby('day', as_index=False)[[cn]].sum()

    nh = df.copy()
    nh['cummax'] = nh[cn].cummax()
    nh = nh.drop_duplicates(subset='cummax', keep='first').reset_index()
    nh['diff'] = (nh['date'].diff() / np.timedelta64(1, 'D')).fillna(0)

    if plot:
        fig, axs = plt.subplots(3, 1, figsize=(10, 10), sharex=True, dpi=200)
        index = df.index
        axs[0].plot(index, values)
        axs[0].set_title('Abstract')
        axs[0].set_ylabel('Daily')
        axs[0].plot(index[window - 1 :], rm)
        axs[1].plot(index, diff)
        axs[1].set_title('Disparity')
        axs[1].set_ylabel('Daily')
        axs[2].plot(index[window - 1 :], detrended)
        axs[2].set_title('Detrended')
        for i in range(3):
            axs[i].yaxis.set_label_position('right')
        plt.suptitle('Decomposition', ha='left', x=0.015, y=1)
        fig.autofmt_xdate()
        plt.savefig("fig/eda_1.png")
        plt.show()

        fig, axs = plt.subplots(2, 1, figsize=(10, 8), dpi=200)
        axs[0].bar(x=countday['day'], height=countday[cn])
        axs[0].set_title('Categorize')
        axs[0].set_ylabel('Total')
        axs[1].hist(x=new[cn], bins=10)
        axs[1].set_title('Distributions')
        axs[1].set_ylabel('Counts')
        for i in range(2):
            axs[i].yaxis.set_label_position('right')
        plt.savefig("fig/eda_2.png")
        plt.show()

        fig, ax = plt.subplots(figsize=(10, 5), dpi=200)
        ax.plot(nh['date'], nh['cummax'])
        for p in range(len(nh)):
            if nh['diff'][p] <= 5.0:
                pass
            else:
                y_ratio = nh['cummax'].max() / 275
                ax.annotate(
                    str(nh['diff'][p]),
                    (mdates.date2num(nh['date'][p]), p),
                    xytext=(-20, nh['cummax'][p] / y_ratio),
                    textcoords='offset pixels',
                )
        ax.set_title('Interval')
        ax.set_ylabel('Cumulative Max')
        ax.yaxis.set_label_position('right')
        plt.savefig("fig/eda_3.png")
        plt.show()


plot_decomposition(df=data, cn='cases', window=12, plot=True)

### Predicting Local<a id='predicting_section'></a>

### Model Selection and Hyperparameters Tuning

In [ ]:
def hyperparameter_tuning(df, cn):
    assert type(cn) == str

    # from sklearn import linear_model, preprocessing, model_selection, pipeline, ensemble, tree, kernel_ridge, neighbors, base

    X = df.drop(columns=cn)
    y = df[cn]
    trainX, testX, trainy, testy = model_selection.train_test_split(
        X, y, test_size=0.8, random_state=7
    )

    lasso_grid = model_selection.GridSearchCV(
        linear_model.Lasso(),
        param_grid={
            'max_iter': [1000, 3000, 5000, 10000, 15000, 30000],
            'alpha': [0.0001, 0.001, 0.01, 0.1, 1],
            'normalize': [True, False],
            'tol': [0.001, 0.01, 0.1, 1],
            'warm_start': [True, False],
            'random_state': [7],
        },
        cv=10,
        scoring='neg_mean_squared_error',
    )

    eas_grid = model_selection.GridSearchCV(
        linear_model.ElasticNet(),
        param_grid={
            'alpha': [1.0, 0.01, 0.1, 0.5, 1],
            'l1_ratio': [0.01, 0.5, 0.1, 1],
            'normalize': [True, False],
            'warm_start': [True, False],
            'random_state': [7],
        },
        cv=10,
        scoring='neg_mean_squared_error',
    )

    gbr_grid = model_selection.GridSearchCV(
        ensemble.GradientBoostingRegressor(),
        param_grid={
            'learning_rate': [0.1, 0.5, 1],
            'n_estimators': [100, 110, 120],
            'min_samples_split': [2, 4, 8],
            'min_samples_leaf': [1, 2, 4],
            'max_depth': [3, 6, 9],
            'max_features': ['auto', 'sqrt', 'log2'],
            'warm_start': [True, False],
            'random_state': [7],
        },
        cv=10,
        scoring='neg_mean_squared_error',
    )

    rfr_grid = model_selection.GridSearchCV(
        ensemble.RandomForestRegressor(),
        param_grid={
            'n_estimators': [110, 120],
            'min_samples_split': [2],
            'min_samples_leaf': [1],
            'max_depth': [6, 9],
            'max_features': ['auto', 'sqrt', 'log2'],
            'warm_start': [True, False],
            'random_state': [7],
        },
        cv=10,
        scoring='neg_mean_squared_error',
    )

    lgb_grid = model_selection.GridSearchCV(
        lgb.LGBMRegressor(),
        param_grid={
            'num_leaves': [10, 31, 50, 75, 150],
            'max_depth': [25, 50, 75],
            'learning_rate': [0.01, 0.1, 0.5, 1],
            'n_estimators': [100, 125, 150],
            'random_state': [7],
        },
        cv=10,
        scoring='neg_mean_squared_error',
    )

    xgb_grid = model_selection.GridSearchCV(
        xgb.XGBRegressor(),
        param_grid={
            'n_estimators': [100, 125, 150],
            'max_depth': [25, 50, 75],
            'learning_rate': [0.01, 0.1, 0.5, 1],
            'min_child_weight': [1, 3, 5],
            'random_state': [7],
        },
        cv=10,
        scoring='neg_mean_squared_error',
    )

    ridge_grid = model_selection.GridSearchCV(
        linear_model.Ridge(),
        param_grid={
            'max_iter': [1000, 3000, 5000, 10000, 15000, 30000],
            'normalize': [True, False],
            'tol': [0.001, 0.01, 0.1, 1],
            'random_state': [7],
        },
        cv=10,
        scoring='neg_mean_squared_error',
    )

    sgd_grid = model_selection.GridSearchCV(
        linear_model.SGDRegressor(),
        param_grid={
            'penalty': ['l1', 'l2', 'elasticnet'],
            'alpha': [0.0001, 0.001, 0.01, 0.1, 1],
            'max_iter': [1000, 3000, 5000, 10000, 15000, 30000],
            'tol': [0.001, 0.01, 0.1, 1],
            'early_stopping': [True],
            'warm_start': [True, False],
            'random_state': [7],
        },
        cv=10,
        scoring='neg_mean_squared_error',
    )

    dt_grid = model_selection.GridSearchCV(
        tree.DecisionTreeRegressor(),
        param_grid={
            'min_samples_split': [2, 4, 5],
            'min_samples_leaf': [1, 2, 3],
            'max_features': ['auto', 'sqrt', 'log2'],
            'max_leaf_nodes': [10, 15, 20, 30],
            'random_state': [7],
        },
        cv=10,
        scoring='neg_mean_squared_error',
    )

    ada_grid = model_selection.GridSearchCV(
        ensemble.AdaBoostRegressor(),
        param_grid={
            'n_estimators': [25, 50, 75, 100],
            'learning_rate': [0.01, 0.1, 1],
            'loss': ['linear', 'square', 'exponential'],
            'random_state': [7],
        },
        cv=10,
        scoring='neg_mean_squared_error',
    )

    kerid_grid = model_selection.GridSearchCV(
        kernel_ridge.KernelRidge(),
        param_grid={'alpha': [0.01, 0.1, 1], 'gamma': [0.001, 0.01, 1]},
    )

    kn_grid = model_selection.GridSearchCV(
        neighbors.KNeighborsRegressor(),
        param_grid={
            'n_neighbors': [5, 10, 15],
            'weights': ['uniform', 'distance'],
            'leaf_size': [5, 10, 20, 30, 40, 50],
        },
    )

    # for m in [lasso_grid, eas_grid, gbr_grid, rfr_grid, lgb_grid, xgb_grid,
    #           ridge_grid, sgd_grid, dt_grid, ada_grid, kerid_grid, kn_grid]:
    #     m.fit(trainX, trainy)
    #     print(m.best_params_)

    # Best params
    lasrs = pipeline.make_pipeline(
        preprocessing.RobustScaler(),
        linear_model.Lasso(
            alpha=0.0001,
            max_iter=10000,
            normalize=False,
            tol=0.001,
            warm_start=True,
            random_state=7,
        ),
    )

    easrs = pipeline.make_pipeline(
        preprocessing.RobustScaler(),
        linear_model.ElasticNet(
            alpha=1.0, l1_ratio=0.01, normalize=False, warm_start=True, random_state=7
        ),
    )

    gbrrs = pipeline.make_pipeline(
        preprocessing.RobustScaler(),
        ensemble.GradientBoostingRegressor(
            max_depth=6,
            max_features='log2',
            min_samples_leaf=1,
            min_samples_split=2,
            n_estimators=110,
            warm_start=True,
            random_state=7,
        ),
    )

    rfrss = pipeline.make_pipeline(
        preprocessing.StandardScaler(),
        ensemble.RandomForestRegressor(
            max_depth=6,
            max_features='log2',
            min_samples_leaf=1,
            min_samples_split=2,
            n_estimators=110,
            warm_start=True,
            random_state=7,
        ),
    )

    lgbmm = pipeline.make_pipeline(
        preprocessing.MinMaxScaler(),
        lgb.LGBMRegressor(
            learning_rate=0.1,
            max_depth=25,
            n_estimators=100,
            num_leaves=10,
            random_state=7,
        ),
    )

    xgbrs = pipeline.make_pipeline(
        preprocessing.RobustScaler(),
        xgb.XGBRegressor(
            learning_rate=0.1,
            max_depth=25,
            min_child_weight=3,
            n_estimators=100,
            random_state=7,
        ),
    )

    ridrs = pipeline.make_pipeline(
        preprocessing.RobustScaler(),
        linear_model.Ridge(max_iter=1000, normalize=False, tol=0.001, random_state=7),
    )

    sgdrs = pipeline.make_pipeline(
        preprocessing.RobustScaler(),
        linear_model.SGDRegressor(
            alpha=0.1,
            early_stopping=True,
            max_iter=1000,
            penalty='elasticnet',
            tol=0.001,
            warm_start=True,
            random_state=7,
        ),
    )

    dtrrs = pipeline.make_pipeline(
        preprocessing.RobustScaler(),
        tree.DecisionTreeRegressor(
            max_features='sqrt',
            max_leaf_nodes=20,
            min_samples_leaf=1,
            min_samples_split=2,
            random_state=7,
        ),
    )

    adars = pipeline.make_pipeline(
        preprocessing.RobustScaler(),
        ensemble.AdaBoostRegressor(
            learning_rate=1, loss='square', n_estimators=25, random_state=7
        ),
    )

    kerrs = pipeline.make_pipeline(
        preprocessing.RobustScaler(), kernel_ridge.KernelRidge(alpha=0.1, gamma=0.001)
    )

    knrrs = pipeline.make_pipeline(
        preprocessing.RobustScaler(),
        neighbors.KNeighborsRegressor(leaf_size=5, n_neighbors=5, weights='distance'),
    )

    model_namelist = [
        'Lasso',
        'ElasticNet',
        'GradientBoosting',
        'RandomForest',
        'LightGBM',
        'XGBoost',
        'Ridge',
        'SGD',
        'DecisionTree',
        'AdaBoost',
        'KernelRidge',
        'KNeighbors',
    ]

    model_list = [
        lasrs,
        easrs,
        gbrrs,
        rfrss,
        lgbmm,
        xgbrs,
        ridrs,
        sgdrs,
        dtrrs,
        adars,
        kerrs,
        knrrs,
    ]
    for n, m in zip(model_namelist, model_list):
        m.fit(trainX, trainy)
        predicted = m.predict(testX)
        mse = np.mean((testy - predicted) ** 2)
        print("Model: {0:20} MSE: {1}".format(n, round(mse, 2)))


hyperparameter_tuning(df=fdata, cn='cases')

In [ ]:
def ml_feature_selection(df, cn, tnf):
    assert type(cn) == str

    X = df.drop(columns=cn)
    y = df[cn]
    trainX, testX, trainy, testy = model_selection.train_test_split(
        X, y, test_size=0.8, random_state=7
    )

    randomforest = ensemble.RandomForestRegressor(
        max_depth=6,
        max_features='log2',
        min_samples_leaf=1,
        min_samples_split=2,
        n_estimators=110,
        warm_start=True,
        random_state=7,
    )
    randomforest.fit(trainX, trainy)
    fi = randomforest.feature_importances_
    indices = np.argsort(fi)
    columns = trainX.columns
    std = np.std([tree.feature_importances_ for tree in randomforest.estimators_])
    print(
        'Explonatory Variables: {} \nResponse Variables: {}\n'.format(
            trainX.columns.tolist(), cn
        )
    )
    print('Feature in Order: ')
    for f in range(trainX.shape[1]):
        print(
            '%d. Feature_no: %d Feature_name: %s (%f)'
            % (f + 1, indices[f], columns[indices[f]], fi[indices[f]])
        )

    feature_selected = columns[indices].tolist()
    feature_selected.insert(0, cn)
    return feature_selected


feature_selected = ml_feature_selection(df=fdata, cn='cases', tnf=3)

### Stacking Model

In [ ]:
class AveragingModels(base.BaseEstimator, base.RegressorMixin, base.TransformerMixin):
    def __init__(self, models):
        self.models = models

    def fit(self, X, y):
        self.models_ = [base.clone(x) for x in self.models]

        for model in self.models_:
            model.fit(X, y)

        return self

    def predict(self, X):
        predictions = np.column_stack([model.predict(X) for model in self.models_])
        return np.mean(predictions, axis=1)


def CombinedModels(df, cn):
    assert type(cn) == str

    lasrs = pipeline.make_pipeline(
        preprocessing.RobustScaler(),
        linear_model.Lasso(
            alpha=0.0001,
            max_iter=10000,
            normalize=False,
            tol=0.001,
            warm_start=True,
            random_state=7,
        ),
    )

    gbrrs = pipeline.make_pipeline(
        preprocessing.RobustScaler(),
        ensemble.GradientBoostingRegressor(
            max_depth=6,
            max_features='log2',
            min_samples_leaf=1,
            min_samples_split=2,
            n_estimators=110,
            warm_start=True,
            random_state=7,
        ),
    )

    rfrss = pipeline.make_pipeline(
        preprocessing.StandardScaler(),
        ensemble.RandomForestRegressor(
            max_depth=6,
            max_features='log2',
            min_samples_leaf=1,
            min_samples_split=2,
            n_estimators=110,
            warm_start=True,
            random_state=7,
        ),
    )

    ridrs = pipeline.make_pipeline(
        preprocessing.RobustScaler(),
        linear_model.Ridge(max_iter=1000, normalize=False, tol=0.001, random_state=7),
    )

    knrrs = pipeline.make_pipeline(
        preprocessing.RobustScaler(),
        neighbors.KNeighborsRegressor(leaf_size=5, n_neighbors=5, weights='distance'),
    )

    kerrs = pipeline.make_pipeline(
        preprocessing.RobustScaler(), kernel_ridge.KernelRidge(alpha=0.1, gamma=0.001)
    )

    model_list = [lasrs, gbrrs, rfrss]

    X = df.drop(columns=cn)
    scaler = preprocessing.MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    y = df[cn]
    trainX, testX, trainy, testy = model_selection.train_test_split(
        X, y, test_size=0.8, random_state=7
    )

    for m in model_list:
        m.fit(trainX, trainy)
        pred = m.predict(testX)

    # Stack Model
    stacked = StackingCVRegressor(
        regressors=(lasrs, gbrrs, rfrss, ridrs, knrrs, kerrs),
        meta_regressor=lasrs,
        use_features_in_secondary=True,
        random_state=7,
    )
    stacked.fit(trainX, trainy)
    stacked_predicted = stacked.predict(testX)
    stacked_mse = np.square(np.subtract(testy, stacked_predicted)).mean()
    print('Stacked MSE:', round(stacked_mse, 2))

    # Blended Model
    blended_model = (
        (0.2 * model_list[0].predict(testX))
        + (0.40 * model_list[1].predict(testX))
        + (0.40 * model_list[2].predict(testX))
    )

    blended_mse = np.square(np.subtract(testy, blended_model)).mean()
    print('Blended MSE:', round(blended_mse, 2))

    # Average Model
    averaged_models = AveragingModels(models=(model_list))
    averaged_models.fit(trainX, trainy)
    averaged_predicted = averaged_models.predict(testX)
    ageraged_mse = np.square(np.subtract(testy, averaged_predicted)).mean()
    print('Averaged MSE:', round(ageraged_mse, 2))

    return stacked


stacked_model = CombinedModels(df=fdata, cn="cases")

Average Model: Lasso, GradientBoosting, RandomForest <br>
Stacked Model: Lasso, GradientBoosting, RandomForest, Ridge, Kneighbors, KernelRidge <br>
Blended Model: 0.2(Lasso), 0.4(GradientBoosting), 0.4(RandomForest) <br>

### Forecast with Lasso Model

In [ ]:
def lasso_model(df, cn, ntf, lag, forecast, plot):
    assert type(cn) == str
    """
    Timeseries to supervised and forecast Lasso
    Parameters:
    -----------
        df: pandas.DataFrame
            The dataframe with explanatory and response variables 
        cn: str
            The columns variables to forecast
        ntf: int
            The number of top feature to select rank by lasso coefficient
        lag: int
            The preceding lag for time series data construction
        forecast: integer
            The time horizon to forecast in days
        plot: boolean, optional
            Visualize plot
    -----------
    Returns:
        - 
    """
    # Preprocessing
    trainX = df.sample(frac=0.8, random_state=7)
    testX = df.drop(trainX.index)
    trainy, testy = trainX.pop(cn), testX.pop(cn)

    l = linear_model.Lasso(
        alpha=0.0001,
        max_iter=10000,
        normalize=False,
        tol=0.001,
        warm_start=True,
        random_state=7,
    )
    l.fit(trainX, trainy)
    pred = l.predict(testX)
    feature_df = pd.DataFrame(
        {'Feature': df.drop(columns=cn).columns, 'Coefficient': l.coef_}
    )
    tf = (
        feature_df.sort_values(by='Coefficient', ascending=False)
        .iloc[:ntf, 0]
        .to_list()
    )

    # Feature selection
    lasso = linear_model.Lasso(
        alpha=0.0001,
        max_iter=10000,
        normalize=False,
        tol=0.001,
        warm_start=True,
        random_state=7,
    )
    lasso.fit(trainX[tf], trainy)
    apred = lasso.predict(testX[tf])

    # Training
    s_feature_model = list()
    for x in range(len(tf)):
        data = df[tf[x]].tolist()
        n_vars = 1 if type(data) is list else data.shape[1]
        temp = pd.DataFrame(data)
        cols, names = list(), list()

        for i in range(lag, 0, -1):  # Input sequence (t-n, ... t-1)
            cols.append(temp.shift(i))
            names += [('%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]
        cols.append(temp.shift(-1))  # Current forecast t
        names += [('%d(t)' % (j + 1)) for j in range(n_vars)]

        new_df = pd.concat(cols, axis=1)
        new_df.columns = names
        new_df.dropna(inplace=True)
        new_df = new_df.add_prefix(str(tf[x]) + ' ')

        XX, yy = new_df.iloc[:, :lag], new_df.iloc[:, lag : lag + 1]
        trainXX = XX.sample(frac=0.8, random_state=7)
        testXX = XX.drop(trainXX.index)
        trainyy = yy.sample(frac=0.8, random_state=7)
        testyy = yy.drop(trainyy.index)
        model = linear_model.Lasso(
            alpha=0.0001,
            max_iter=10000,
            normalize=False,
            tol=0.001,
            warm_start=True,
            random_state=7,
        )
        model.fit(trainXX, trainyy)
        s_feature_model.append(model)
        # print(np.sqrt(np.mean(testyy.values.flatten()-model.predict(testXX))**2))

    # Forecast
    result = df[tf].iloc[-lag:, :].copy()
    # display(result)
    result['Forecast'] = 0
    for _ in range(forecast):
        temp_result = list()
        for x in range(len(s_feature_model)):
            temp_result.append(
                int(
                    s_feature_model[x].predict(
                        result.iloc[-lag:, x].values.reshape(1, -1)
                    )
                )
            )
        temp_result.append(int(lasso.predict(np.array(temp_result).reshape(1, -1))))
        result = result.append(
            pd.Series(temp_result, index=result.columns), ignore_index=True
        )

    moving_average_window = 7
    result['Forecast Moving Average'] = (
        result['Forecast'].rolling(window=moving_average_window).mean()
    )

    # Output
    # feature_df.plot.bar(x='Feature', y='Coefficient')
    # print('Without feature Selection: {}'.format(np.sqrt(np.mean(testy-pred)**2)))
    # print('With feature selection: {}'.format(np.sqrt(np.mean(testy-apred)**2)))
    # print('Forecast: {} | Lag: {}'.format(forecast, lag))

    if plot:
        fig, ax = plt.subplots(dpi=200)
        plt.plot(df[cn])
        plt.plot(
            pd.Series(
                result['Forecast'][-forecast:].tolist(),
                index=np.arange(
                    int(df.index[-1:].values[0]),
                    int(df.index[-1:].values[0]) + forecast,
                ),
            )
        )
        plt.plot(
            pd.Series(
                result['Forecast Moving Average'][
                    -forecast + moving_average_window :
                ].tolist(),
                index=np.arange(
                    int(df.index[-1:].values[0]) + moving_average_window,
                    int(df.index[-1:].values[0]) + forecast,
                ),
            )
        )
        plt.legend(
            ['Actual', 'Forecast', 'Forecast Moving Average'], ncol=5, loc='upper left'
        )
        plt.title(
            'Feature Selected: {} | Forecast: {} | Lag: {}'.format(ntf, forecast, lag),
            color='#787878',
        )
        plt.suptitle(
            'Projection: {} Forward Forecast'.format(cn),
            ha='left',
            x=0.015,
            y=0.95,
            fontsize=20,
        )
        plt.figtext(0.85, 0.9, 'Lasso Model', ha='left')
        ax.set_xlabel('Range')
        ax.set_ylabel('Values')
        ax.yaxis.set_label_position('right')
        plt.show()

In [ ]:
# lasso_model(df=fdata, cn='cases', ntf=3, lag=7, forecast=90, plot=True)
# lasso_model(df=fdata, cn='cases', ntf=3, lag=30, forecast=90, plot=True)

### Forecast with Stacked Model

In [ ]:
"def stacked_model(df, cn, lag, forecast, plot):
    assert type(cn) == str
    """
    Timeseries to supervised and forecast Lasso
    Parameters:
    -----------
        df: pandas.DataFrame
            The dataframe with explanatory and response variables 
        cn: str
            The columns variables to forecast
        lag: int
            The preceding lag for time series data construction
        forecast: integer
            The time horizon to forecast in days
        plot: boolean, optional
            Visualize plot
    -----------
    Returns:
        - 
    """
    # Preprocessing
    trainX = df.sample(frac=0.8, random_state=7)
    testX = df.drop(trainX.index)
    trainy, testy = trainX.pop(cn), testX.pop(cn)
    tf = trainX.columns.tolist()

    lasrs = pipeline.make_pipeline(
        preprocessing.RobustScaler(),
        linear_model.Lasso(
            alpha=0.0001,
            max_iter=10000,
            normalize=False,
            tol=0.001,
            warm_start=True,
            random_state=7,
        ),
    )

    gbrrs = pipeline.make_pipeline(
        preprocessing.RobustScaler(),
        ensemble.GradientBoostingRegressor(
            max_depth=6,
            max_features='log2',
            min_samples_leaf=1,
            min_samples_split=2,
            n_estimators=110,
            warm_start=True,
            random_state=7,
        ),
    )

    rfrss = pipeline.make_pipeline(
        preprocessing.StandardScaler(),
        ensemble.RandomForestRegressor(
            max_depth=6,
            max_features='log2',
            min_samples_leaf=1,
            min_samples_split=2,
            n_estimators=110,
            warm_start=True,
            random_state=7,
        ),
    )
    ridrs = pipeline.make_pipeline(
        preprocessing.RobustScaler(),
        linear_model.Ridge(max_iter=1000, normalize=False, tol=0.001, random_state=7),
    )

    knrrs = pipeline.make_pipeline(
        preprocessing.RobustScaler(),
        neighbors.KNeighborsRegressor(leaf_size=5, n_neighbors=5, weights='distance'),
    )

    kerrs = pipeline.make_pipeline(
        preprocessing.RobustScaler(), kernel_ridge.KernelRidge(alpha=0.1, gamma=0.001)
    )

    stacked_model = StackingCVRegressor(
        regressors=(lasrs, gbrrs, rfrss, ridrs, knrrs, kerrs),
        meta_regressor=lasrs,
        use_features_in_secondary=True,
        random_state=7,
    )

    stacked_model.fit(trainX, trainy)
    pred = stacked_model.predict(testX)

    # Training
    model_list = list()
    for x in range(len(tf)):
        data = df[tf[x]].tolist()
        n_vars = 1 if type(data) is list else data.shape[1]
        temp = pd.DataFrame(data)
        cols, names = list(), list()

        for i in range(lag, 0, -1):  # Input sequence (t-n, ... t-1)
            cols.append(temp.shift(i))
            names += [('%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]
        cols.append(temp.shift(-1))  # Current forecast t
        names += [('%d(t)' % (j + 1)) for j in range(n_vars)]

        new_df = pd.concat(cols, axis=1)
        new_df.columns = names
        new_df.dropna(inplace=True)
        new_df = new_df.add_prefix(str(tf[x]) + ' ')

        XX, yy = new_df.iloc[:, :lag], new_df.iloc[:, lag : lag + 1]
        trainXX = XX.sample(frac=0.8, random_state=7)
        testXX = XX.drop(trainXX.index)
        trainyy = yy.sample(frac=0.8, random_state=7)
        testyy = yy.drop(trainyy.index)
        model = StackingCVRegressor(
            regressors=(lasrs, gbrrs, rfrss, ridrs, knrrs, kerrs),
            meta_regressor=lasrs,
            use_features_in_secondary=True,
            random_state=7,
        )
        model.fit(trainXX, trainyy)
        model_list.append(model)
        # print(np.sqrt(np.mean(testyy.values.flatten()-model.predict(testXX))**2))

    # Forecast
    result = df[tf].iloc[-lag:, :].copy()
    # display(result)
    result['Forecast'] = 0
    for _ in range(forecast):
        temp_result = list()
        for x in range(len(model_list)):
            temp_result.append(
                int(model_list[x].predict(result.iloc[-lag:, x].values.reshape(1, -1)))
            )
        temp_result.append(
            int(stacked_model.predict(np.array(temp_result).reshape(1, -1)))
        )
        result = result.append(
            pd.Series(temp_result, index=result.columns), ignore_index=True
        )

    moving_average_window = 7
    result['Forecast Moving Average'] = (
        result['Forecast'].rolling(window=moving_average_window).mean()
    )

    # Output
    if plot:
        fig, ax = plt.subplots(dpi=200)
        plt.plot(df[cn])
        plt.plot(
            pd.Series(
                result['Forecast'][-forecast:].tolist(),
                index=np.arange(
                    int(df.index[-1:].values[0]),
                    int(df.index[-1:].values[0]) + forecast,
                ),
            )
        )
        plt.plot(
            pd.Series(
                result['Forecast Moving Average'][
                    -forecast + moving_average_window :
                ].tolist(),
                index=np.arange(
                    int(df.index[-1:].values[0]) + moving_average_window,
                    int(df.index[-1:].values[0]) + forecast,
                ),
            )
        )
        plt.legend(
            ['Actual', 'Forecast', 'Forecast Moving Average'], ncol=5, loc='upper left'
        )
        plt.title("Forecast: {} | Lag: {}".format(forecast, lag), color='#787878')
        plt.suptitle(
            'Projection: {} Forward Forecast'.format(cn),
            ha='left',
            x=0.015,
            y=0.95,
            fontsize=20,
        )
        plt.figtext(0.85, 0.9, 'Stacked Model', ha='left')
        ax.set_xlabel('Range')
        ax.set_ylabel('Values')
        ax.yaxis.set_label_position('right')
        plt.savefig(
            "fig/stacked_model_local_{}_{}_{}_{}".format(
                len(df.columns), cn, str(lag), str(forecast)
            )
        )
        plt.show()

In [ ]:
number_of_features = 3
stacked_model(df=fdata, cn='cases', lag=14, forecast=90, plot=True)
stacked_model(
    df=fdata[feature_selected[: number_of_features + 1]],
    cn='cases',
    lag=14,
    forecast=90,
    plot=True,
)

### Forecast with Long Short Term Memory Model

In [ ]:
def lstm_model(df, cn, window, forecast, epochs, bs, n_samples=1, seed=True, plot=True):
    assert type(cn) == str
    """
    Train and forecast LSTM. 
    Parameters:
    -----------
        df: pandas.DataFrame
            The Dataframe with datetime.date as index with explanatory and response variables  
        cn: str
            The columns variables to forecast
        window: int
            The preceding window size for time series data construction
        forecast: int
            The time horizon to forecast
        epochs: int
            The number of training epochs
        bs: int
            The number of training batch_size
        n_samples: int, default=1
            The number of model created for computing confidence interval
        plot: boolean, default=True, optional
            Visualize plot
    ----------
    Returns: 
        - 
    """
    # Preprocessing
    data = df.filter([cn]).values
    nmin, nmax = data.min(), data.max()
    scaled_data = ((df[cn] - nmin) / (nmax - nmin)).values.reshape(-1, 1)
    split = round(0.8 * len(data))
    train = scaled_data[:split]
    trainX, trainy = [], []
    for i in range(window, len(train)):
        trainX.append(train[i - window : i, 0])
        trainy.append(train[i, 0])
    trainX, trainy = np.array(trainX), np.array(trainy)
    trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))

    tf.random.set_seed(7)  # set seed for reproducibility
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='loss', min_delta=0.003, patience=3, restore_best_weights=True
    )

    # Training
    lstm_input = tf.keras.layers.Input(shape=(trainX.shape[1], 1))
    lstm_1 = tf.keras.layers.LSTM(units=50, return_sequences=True)(lstm_input)
    lstm_2 = tf.keras.layers.LSTM(units=150)(lstm_1)
    lstm_output = tf.keras.layers.Dense(units=1)(lstm_2)
    model = tf.keras.models.Model(inputs=lstm_input, outputs=lstm_output)
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(
        trainX,
        trainy,
        epochs=epochs,
        batch_size=bs,
        verbose=0,
        callbacks=[early_stopping],
    )

    # Testing
    test = scaled_data[split - window :]
    testX, testy = [], data[split:]
    for i in range(window, len(test)):
        testX.append(test[i - window : i, 0])
    testX = np.array(testX)
    testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))
    prediction = model.predict(testX)
    prediction = (nmax - nmin) * prediction + nmin
    prediction_series = pd.Series(prediction.flatten(), index=df.index[split:])

    # Forecast
    results = pd.DataFrame()
    for n in range(n_samples):
        forecast_list = scaled_data[-window:]
        lstm_input = tf.keras.layers.Input(shape=(trainX.shape[1], 1))
        lstm_1 = tf.keras.layers.LSTM(units=50, return_sequences=True)(lstm_input)
        lstm_2 = tf.keras.layers.LSTM(units=150)(lstm_1)
        lstm_output = tf.keras.layers.Dense(units=1)(lstm_2)
        model = tf.keras.models.Model(inputs=lstm_input, outputs=lstm_output)
        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(
            trainX,
            trainy,
            epochs=epochs,
            batch_size=bs,
            verbose=0,
            callbacks=[early_stopping],
        )
        for _ in range(forecast):
            x = forecast_list[-window:]
            x = x.reshape((1, window, 1))
            out = model.predict(x)[0][0]
            forecast_list = np.append(forecast_list, out)

        forecast_list = forecast_list[window - 1 :]
        forecast_list = (nmax - nmin) * forecast_list + nmin
        results['Forecast{}'.format(n + 1)] = forecast_list.copy()

    last_date = df.index.values[-1]
    forecast_dates = pd.date_range(last_date, periods=forecast + 1).tolist()
    results['Date'] = forecast_dates
    results.set_index('Date', inplace=True)
    results['Mean'] = results.mean(axis=1)
    results['_CI'] = results['Mean'] - results.std(axis=1) / np.sqrt(n_samples) * 1.96
    results['+CI'] = results['Mean'] + results.std(axis=1) / np.sqrt(n_samples) * 1.96

    # Output
    # print('Window: {} | Forecast: {} | Epochs: {} | Batch Size: {} | N_samples: {}'\
    #       .format(window, forecast, epochs, bs, n_samples))
    # print('RMSE:', np.sqrt(np.mean(testy-prediction)**2))

    if plot:
        fig, ax = plt.subplots(dpi=200)
        plt.plot(df[cn][:split])
        plt.plot(df[cn][split:])
        plt.plot(prediction_series)
        plt.plot(results['Mean'])
        plt.fill_between(results.index, results['_CI'], results['+CI'], color='#ababab')
        plt.legend(
            ['Train', 'Test', 'Prediction', 'Forecast', 'CI'], ncol=5, loc='upper left'
        )
        plt.title(
            'Window: {} | Forecast: {} | Epochs: {} | Batch Size: {} | N_samples: {} \nRMSE: {}'.format(
                window,
                forecast,
                epochs,
                bs,
                n_samples,
                np.round(np.sqrt(np.mean(testy - prediction) ** 2)),
                4,
            ),
            color='grey',
        )
        plt.suptitle(
            'Projection: {} Forward Forecast'.format(cn), ha='left', x=0.015, y=0.95
        )
        plt.figtext(0.85, 0.9, 'LSTM Model', ha='left')
        fig.autofmt_xdate()
        ax.set_xlabel('Date')
        ax.set_ylabel('Values')
        ax.yaxis.set_label_position('right')
        plt.savefig(
            "fig/lstm_model_local_{}_{}_{}_{}_{}".format(
                cn, str(window), str(forecast), str(epochs), str(bs)
            )
        )
        plt.show()

In [ ]:
lstm_model(df=data, cn='cases', window=45, forecast=90, epochs=1, bs=8, plot=True)
lstm_model(df=data, cn='cases', window=30, forecast=90, epochs=1, bs=16, plot=True)

## Global<a id='global_view'></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from cycler import cycler

custom_style = {
    'figure.autolayout': True,
    'figure.titlesize': 20,
    'figure.figsize': (10, 5),
    'figure.dpi': 100,
    'axes.spines.top': False,
    'axes.spines.left': False,
    'axes.titlesize': 10,
    'axes.titlelocation': 'left',
    'axes.labelsize': 14,
    'axes.grid': True,
    'axes.prop_cycle': cycler(
        color=['#d73027', '#00518b', '#b1ef89', '#ffd500', '#000000']
    ),
    'grid.color': '#969696',
    'xtick.direction': 'inout',
    'ytick.direction': 'inout',
    'xtick.minor.visible': True,
    'ytick.minor.visible': True,
    'ytick.right': True,
    'ytick.left': False,
    'ytick.labelright': True,
    'ytick.labelleft': False,
    'xaxis.labellocation': 'right',
    'yaxis.labellocation': 'top',
    'font.family': 'monospace',
    'legend.fontsize': 10,
    'legend.loc': 'best',
}

plt.style.use(custom_style)

In [ ]:
%%time
all = pd.read_csv("https://covid.ourworldindata.org/data/owid-covid-data.csv")
all = all[all['location'] == 'World'][
    ['date', 'total_cases', 'new_cases', 'total_deaths', 'new_deaths']
].reset_index(drop=True)
all.columns = ['date', 'total_cases', 'new_cases', 'total_deceased', 'new_deceased']
all = all.iloc[70:]
all['date'] = pd.to_datetime(all['date'])
all.set_index('date', inplace=True)

In [ ]:
with plt.style.context('dark_background'):
    fig, ax = plt.subplots(dpi=200)
    plt.plot(all['total_cases'], label='Total Cases', linewidth=2)
    plt.plot(all['new_cases'], label='New Cases', linewidth=2)
    plt.plot(all['total_deceased'], label='Total Deceased', linewidth=2)
    plt.plot(all['new_deceased'], label='New Deceased', linewidth=2)
    ax.set_yscale('log')
    plt.legend()
    plt.ylabel('Log Scale')
    ax.yaxis.set_label_position('right')
    plt.xlabel('Date')
    plt.suptitle('Global Outlook', ha='left', x=0.015, y=0.95)
    plt.savefig("fig/global_outlook.png")
    plt.show()

In [ ]:
def lstm_outbreak_model(df, cn, window, forecast, epochs, bs, seed=True, plot=True):
    # Preprocessing
    data = df.filter([cn]).values
    nmin, nmax = data.min(), data.max()
    scaled_data = ((df[cn] - nmin) / (nmax - nmin)).values.reshape(-1, 1)
    split = round(0.8 * len(data))
    train = scaled_data[:split]
    trainX, trainy = [], []
    for i in range(window, len(train)):
        trainX.append(train[i - window : i, 0])
        trainy.append(train[i, 0])
    trainX, trainy = np.array(trainX), np.array(trainy)
    trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))

    tf.random.set_seed(7)  # set seed for reproducibility
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='loss', min_delta=0.003, patience=3, restore_best_weights=True
    )

    # Training
    lstm_input = tf.keras.layers.Input(shape=(trainX.shape[1], 1))
    lstm_1 = tf.keras.layers.LSTM(units=50, return_sequences=True)(lstm_input)
    lstm_2 = tf.keras.layers.LSTM(units=150)(lstm_1)
    lstm_output = tf.keras.layers.Dense(units=1)(lstm_2)
    model = tf.keras.models.Model(inputs=lstm_input, outputs=lstm_output)
    model.compile(
        optimizer='adam',
        loss='mean_squared_error',
        metrics=[
            tf.keras.metrics.MeanSquaredError(name='mse'),
            tf.keras.metrics.MeanAbsoluteError(name='mae'),
        ],
    )
    model.fit(
        trainX,
        trainy,
        epochs=epochs,
        batch_size=bs,
        verbose=1,
        callbacks=[early_stopping],
    )

    # Testing
    test = scaled_data[split - window :]
    testX, testy = [], data[split:]
    for i in range(window, len(test)):
        testX.append(test[i - window : i, 0])
    testX = np.array(testX)
    testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))
    prediction = model.predict(testX)
    prediction = (nmax - nmin) * prediction + nmin
    prediction_series = pd.Series(prediction.flatten(), index=df.index[split:])

    # Forecast
    forecast_list = scaled_data[-window:]
    for _ in range(forecast):
        x = forecast_list[-window:]
        x = x.reshape((1, window, 1))
        out = model.predict(x)[0][0]
        forecast_list = np.append(forecast_list, out)

    forecast_list = forecast_list[window - 1 :]
    forecast_list = (nmax - nmin) * forecast_list + nmin

    last_date = df.index.values[-1]
    forecast_dates = pd.date_range(last_date, periods=forecast + 1).tolist()

    results = pd.DataFrame()
    results['Forecast'] = forecast_list
    results['Date'] = forecast_dates
    results.set_index('Date', inplace=True)

    if plot:
        fig, ax = plt.subplots(figsize=(10, 7), dpi=200)
        plt.plot(df[cn][:split])
        plt.plot(df[cn][split:])
        plt.plot(prediction_series)
        plt.plot(results['Forecast'])
        plt.legend(
            ['Train', 'Test', 'Prediction', 'Forecast', 'CI'], ncol=5, loc='upper left'
        )
        plt.title(
            'Window: {} | Forecast: {} | Epochs: {} | Batch Size: {} | RMSE: {}'.format(
                window,
                forecast,
                epochs,
                bs,
                np.round(np.sqrt(np.mean(testy - prediction) ** 2)),
                4,
            ),
            color='grey',
        )
        plt.suptitle(
            'Projection: {} Forward Forecast'.format(cn),
            ha='left',
            x=0.015,
            y=1,
            fontsize=18,
        )
        plt.figtext(0.85, 0.9, 'LSTM Model', ha='left')
        fig.autofmt_xdate()
        ax.set_xlabel('Date')
        ax.set_ylabel('Cases')
        ax.yaxis.set_label_position('right')
        plt.savefig(
            "fig/lstm_model_global_{}_{}_{}_{}_{}".format(
                cn, str(window), str(forecast), str(epochs), str(bs)
            )
        )
        plt.show()

In [ ]:
lstm_outbreak_model(
    df=all, cn='new_cases', window=14, forecast=90, epochs=1, bs=8, plot=True
)
lstm_outbreak_model(
    df=all, cn='new_cases', window=45, forecast=90, epochs=1, bs=8, plot=True
)

In [ ]:
lstm_outbreak_model(
    df=all, cn='new_deceased', window=14, forecast=90, epochs=1, bs=8, plot=True
)
lstm_outbreak_model(
    df=all, cn='new_deceased', window=45, forecast=90, epochs=1, bs=8, plot=True
)

# Dependencies<a id='dependencies'></a>

In [ ]:
# !pip install matplotlib==3.5.1
# !pip install pandas==1.3.5
# !pip install scipy==1.7.1
# !pip install lightgbm==3.3.1
# !pip install tensorflow==2.7.0
# !pip install numpy==1.20.0
# !pip install numba==0.50.1
# !pip install xgboost==1.5.1
# !pip install sklearn==0.23.2
# !pip install mlxtend==0.19.0